In [1]:
from keras.models import load_model
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
# This function will convert the timeseries data to a supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [3]:
model = load_model('model.h5')

In [4]:
df = pd.read_csv('test_data.csv', index_col = 0)
df.head()

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
1/1/14 0:00,24,-20,7.0,1014,NW,143.48,0,0
1/1/14 1:00,53,-20,7.0,1013,NW,147.50,0,0
1/1/14 2:00,65,-20,6.0,1013,NW,151.52,0,0
1/1/14 3:00,70,-20,6.0,1013,NW,153.31,0,0
1/1/14 4:00,79,-18,3.0,1012,cv,0.89,0,0


In [5]:
values = df.values
# Encode the categorical wnd_dir column with LabelEncoder 
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
values = values.astype('float32')
# Normalize the values by scaling
scaler = MinMaxScaler(feature_range = (0,1))
scaled = scaler.fit_transform(values)
# Convert the timeseries data into supervised learning
reframed = series_to_supervised(scaled)
# Drop non-required columns
reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var1(t)
1,0.035768,0.307692,0.363636,0.456522,0.333333,0.324530,0.0,0.0,0.078987
2,0.078987,0.307692,0.363636,0.434784,0.333333,0.333651,0.0,0.0,0.096870
3,0.096870,0.307692,0.345455,0.434784,0.333333,0.342772,0.0,0.0,0.104322
4,0.104322,0.307692,0.345455,0.434784,0.333333,0.346834,0.0,0.0,0.117735
5,0.117735,0.338462,0.290909,0.413044,1.000000,0.000998,0.0,0.0,0.137109


In [6]:
# Save last column as target y values
values = reframed.values
target = values[:,-1]
X = values[:, : -1]

In [7]:
X = X.reshape((X.shape[0], 1, X.shape[1]))
print(X.shape)

(8759, 1, 8)


In [8]:
predicted = model.predict(X)
r2_score(target,predicted)

0.93188505451846426